In [14]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time

In [15]:
url = "http://www.indeed.com/jobs"

In [16]:
response = requests.get(url)

In [17]:
soup = BeautifulSoup(response.text, 'html5lib')

In [18]:
DEFAULT_CITIES = {'New York', 'Chicago', 'San Francisco', 'Austin', 'Seattle',
                  'Los Angeles', 'Philadelphia', 'Atlanta', 'Dallas',
                  'Pittsburgh', 'Portland', 'Phoenix', 'Denver', 'Houston',
                  'Miami'}

YOUR_CITIES = {'Boston', 'Washington DC', 'St Louis', 'San Diego',
               'San Antonio', 'Columbus', 'Sacramento', 'Charlotte', 'Memphis',
               'Detroit', 'Nashville', 'Jacksonville', 'Indianapolis',
               'Fort Worth', 'Charlotte', 'El Paso', 'Oklahoma City',
               'Las Vegas', 'Louisville', 'Milwaukee', 'Albuquerque', 'Tucson',
               'Kansas City', 'Mesa', 'Colorado Springs', 'Raleigh', 'Omaha',
               'Virginia Beach', 'Minneapolis', 'New Orleans', 'Tampa',
               'San Jose', 'Baltimore', 'Fresno', 'Oakland', 'Tulsa', 'Madison',
               'Arlington', 'Wichita', 'Cleveland', 'Aurora', 'Honolulu',
               'Orlando', 'Anchorage', 'Des Moines', 'Salt Lake City',
               'Lexington', 'Cincinnati', 'Newark', 'Durham', 'Buffalo',
               'Baton Rouge', 'Richmond', 'Boise', 'Birmingham', 'Little Rock',
               'Grand Rapids', 'Worcester', 'Providence', 'Sioux Falls',
               'Jackson', 'Hartford', 'Bridgeport', 'Jersey City', 'Charleston',
               'Billings', 'Fargo', 'Augusta'}


In [19]:
print(soup.prettify())

<!DOCTYPE html>
<html dir="ltr" lang="en">
 <head>
  <script id="polyfill-script-bundle">
   /* Disable minification (remove `.min` from URL path) for more info */

(function(self, undefined) {function ArrayCreate(r){if(1/r==-Infinity&&(r=0),r>Math.pow(2,32)-1)throw new RangeError("Invalid array length");var n=[];return n.length=r,n}function Call(t,l){var n=arguments.length>2?arguments[2]:[];if(!1===IsCallable(t))throw new TypeError(Object.prototype.toString.call(t)+"is not a function.");return t.apply(l,n)}function Get(n,t){return n[t]}function HasOwnProperty(r,t){return Object.prototype.hasOwnProperty.call(r,t)}function HasProperty(n,r){return r in n}function IsArray(r){return"[object Array]"===Object.prototype.toString.call(r)}function IsCallable(n){return"function"==typeof n}function RequireObjectCoercible(e){if(null===e||e===undefined)throw TypeError();return e}function SameValueNonNumber(e,n){return e===n}function ToBoolean(o){return Boolean(o)}function ToInteger(n){var i=Number(

In [20]:
params = {'q': 'data scientist', 'radius': '100'}
max_results = 100

In [21]:
def _location(result):
    try:
        location = result.find('span', class_='location').get_text().strip()
        return location
    except:
        return None

In [22]:
def _company(result):
    try:
        company = result.find('span', class_='company').get_text().strip()
        return company
    except:
        return None

In [23]:
def _title(result):
    try:
        title = result.find('a', attrs={'data-tn-element': "jobTitle"}).get(
            'title')
        return title
    except:
        return None

In [24]:
def _salary(result):
    try:
        company = result.find('span', class_='salaryText').get_text().strip()
        return company
    except:
        return None

In [13]:
scraped_data = {'location': [],
            'company': [],
            'title': [],
            'salary': []}

for city in DEFAULT_CITIES:
    for start in range(0, max_results, 15):
        url_params = params.copy()
        url_params.update({'l': city, 'start': start})

        response = requests.get(url, params=url_params)
        soup = BeautifulSoup(response.text, 'lxml')
        results = soup.find_all('div', class_='result')
        
        time.sleep(0.5)
        print(response.url)
        
        for result in results:
            scraped_data['location'].append(_location(result))
            scraped_data['company'].append(_company(result))
            scraped_data['title'].append(_title(result))
            scraped_data['salary'].append(_salary(result))
company_info_df = pd.DataFrame(scraped_data)


https://www.indeed.com/jobs?q=data+scientist&radius=100&l=Chicago&start=0
https://www.indeed.com/jobs?q=data+scientist&radius=100&l=Chicago&start=15
https://www.indeed.com/jobs?q=data+scientist&radius=100&l=Chicago&start=30
https://www.indeed.com/jobs?q=data+scientist&radius=100&l=Chicago&start=45
https://www.indeed.com/jobs?q=data+scientist&radius=100&l=Chicago&start=60
https://www.indeed.com/jobs?q=data+scientist&radius=100&l=Chicago&start=75
https://www.indeed.com/jobs?q=data+scientist&radius=100&l=Chicago&start=90
https://www.indeed.com/jobs?q=data+scientist&radius=100&l=New+York&start=0
https://www.indeed.com/jobs?q=data+scientist&radius=100&l=New+York&start=15
https://www.indeed.com/jobs?q=data+scientist&radius=100&l=New+York&start=30
https://www.indeed.com/jobs?q=data+scientist&radius=100&l=New+York&start=45
https://www.indeed.com/jobs?q=data+scientist&radius=100&l=New+York&start=60
https://www.indeed.com/jobs?q=data+scientist&radius=100&l=New+York&start=75
https://www.indeed.co

In [27]:
company_df = company_info_df.drop_duplicates()
company_df

,location,company,title,salary
0,None,Hirsch Law Group,Court Records Researcher/Data Miner - (Chicago),$15 an hour
1,None,Society for Vascular Surgery,Statistician (part-time),None
2,None,i-Pharm Consulting,Sr. Statistical Programmer - (100% REMOTE) - E...,"$115,000 - $140,000 a year"
3,"Chicago, IL",Cloudbigdata Technologies,Entry level Data Scientist,None
4,"Chicago, IL 60606 (The Loop area)",Saggezza,Data Scientist,None
...,...,...,...,...
1409,"Mountain View, CA",Bytedance,Data Scientist,None
1410,"Sacramento, CA 95814 (Mansion Flats area)",Capio Group,Data Scientist - 27877227,None
1411,"Sunnyvale, CA 94085",Proofpoint,Data Science Intern - Undergrad,None
1412,"San Jose, CA",Jabil Circuit,Data Scientist,None


In [25]:
company_info_df2 = company_info_df.loc[company_info_df['salary'].notnull()]
company_info_df2.drop_duplicates(subset=['company',
                               'location', 'salary', 'title'],
                       keep='last', inplace=True)
company_info_df2

C:\Users\totor\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,location,company,title,salary
10,"Chicago, IL 60601 (The Loop area)",Hale Testing Solutions,Machine Learning/Data Scientist,"$87,829 - $122,372 a year"
71,None,Hirsch Law Group,Court Records Researcher/Data Miner - (Chicago),$15 an hour
128,"New York, NY 10036 (Garment District area)",Ampersand,Data Science Engineer,"$100,000 - $200,000 a year"
224,"Seattle, WA",City of Seattle,Policy and Statistical Analyst,"$85,629 - $128,433 a year"
263,"Seattle, WA 98101 (Downtown area)",Liberty Mutual Insurance,"Director II, Data Science","$153,300 - $201,600 a year"
273,"Seattle, WA 98122 (Central District area)",Johns Hopkins University,Sr. Research Data Analyst,"$52,495 - $72,210 a year"
277,None,Québec Immigration Office,Senior Machine Learning Developer (visa sponso...,$55 - $67 an hour
295,"Philadelphia, PA 19103",Boxplot,Data Scientist,"$30,000 a year"
317,None,North America Service Center for Chinese Stude...,房产咨询顾问/Real Estate Consultant/ Real Estate Sal...,"$80,000 - $150,000 a year"
325,"Wilmington, DE","Canopy One Solutions, Inc.",Data Scientist,$55 - $65 an hour


In [18]:
company_info_df3= company_info_df2.loc[company_info_df2['location'].notnull()]
company_info_df3

,location,company,title,salary
13,"Alpharetta, GA",Quest America,Data Scientist,$65 - $70 an hour
32,"Alpharetta, GA",Quest America,Data Scientist,$65 - $70 an hour
51,"Alpharetta, GA",Quest America,Data Scientist,$65 - $70 an hour
69,"Alpharetta, GA",Quest America,Data Scientist,$65 - $70 an hour
88,"Alpharetta, GA",Quest America,Data Scientist,$65 - $70 an hour
...,...,...,...,...
14897,"Alpharetta, GA",Quest America,Data Scientist,$65 - $70 an hour
14916,"Alpharetta, GA",Quest America,Data Scientist,$65 - $70 an hour
14935,"Alpharetta, GA",Quest America,Data Scientist,$65 - $70 an hour
14950,"Alpharetta, GA",Quest America,Data Scientist,$65 - $70 an hour


In [28]:
company_df.to_csv('scrape_indeed_data_scientist.csv', index=False, encoding='utf8')